# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [3]:
# Check if g_key imported properly
print()

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
file = 'city_data.csv'

df = pd.read_csv(file)

df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ribeira grande,21,PT,1597885901,68,38.52,-28.70,68.90,15.59
1,mataura,93,NZ,1597885901,77,-46.19,168.86,55.99,1.99
2,chokurdakh,100,RU,1597885790,93,70.63,147.92,36.72,9.73
3,stornoway,100,GB,1597885797,87,58.21,-6.39,59.00,14.99
4,vaini,90,TO,1597885903,83,-21.20,-175.20,77.00,10.29


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [10]:
locations = df[['Lat', 'Lng']].astype(float)
humidity = df['Humidity']

# Create poverty heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100, point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
# Narrow to temp between 70 and 80, wind speed less than 10, and zero cloudiness
# Drop NaN as well
drop_temp = df.loc[(df['Max Temp'] < 80) & (df['Max Temp'] > 70)]
drop_wind = drop_temp.loc[drop_temp['Wind Speed'] < 10]
zero_cloud = drop_wind.loc[drop_wind['Cloudiness'] == 0]
narrow_df = zero_cloud.dropna()

narrow_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
5,mao,0,ES,1597885904,88,39.89,4.27,75.20,4.70
18,mabaruma,0,GY,1597885912,88,8.20,-59.78,76.44,2.64
44,carlyle,0,US,1597885927,63,38.61,-89.37,73.00,7.34
47,mackay,0,AU,1597885930,47,-21.15,149.20,78.80,5.82
65,san giovanni in fiore,0,IT,1597885937,73,39.26,16.70,73.40,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [56]:
# Store into hotel_df
hotel_df = narrow_df

# Add hotel name column
hotel_df['Hotel Name'] = ''

# Set up params
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Make dictionary to hold latitude and longitude data of each city
# Used https://stackoverflow.com/questions/26716616/convert-a-pandas-dataframe-to-a-dictionary as reference
hotel_df_slice = hotel_df[['City', 'Lat', 'Lng']]
hotel_df_slice.set_index('City', drop = True, inplace = True)
dictionary = hotel_df_slice.to_dict(orient = 'index')

hotels = []

# For each city in dictionary, find first hotel using Google Places API
for row in dictionary:
    
    # Get latitude and longitude
    lat = dictionary[row]['Lat']
    long = dictionary[row]['Lng']

    target_coordinates = str(lat) + ', ' + str(long)
    
    # Add these coords to params
    params['location'] = target_coordinates
    
    # Run a request using params dictionary
    response = requests.get(base_url, params=params)
    response = response.json()
    
    # Get first hotel name and store in hotel list
    try:
        hotels.append(response['results'][0]['name'])
    except:
        pass

In [62]:
print(hotels)
len(hotels)

hotels.append('Dummy Value')

['ARTIEM Capri', 'Centerstone Inn & Suites - Carlyle', 'International Lodge Motel', 'Casa Federico B&B', 'Floresta Amazonica Hotel', 'ARTIEM Capri', 'רגע בנחת', 'Dena Hotel', 'Pilões Palace Hotel', 'Park hotel', 'Valle Aridane', 'Areti Suites', 'Ikathouse', 'Belchior Palace Hotel', 'Provost Motor Inn', 'Hotel Mineiro Ltda', 'محل الشريف محمد العايب', 'Hacienda Barriche', 'Montes Altos', 'LE SAMANA', 'Il Veliero', 'فندق التوجان', 'Thank Inn Chain Hotel gansu wuwei liangzhou district xiguan middle road', 'هتل آپارتمان آرامیس', 'محطة التلفاز']


In [64]:
hotel_df['Hotel Name'] = hotels
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
5,mao,0,ES,1597885904,88,39.89,4.27,75.20,4.70,ARTIEM Capri
18,mabaruma,0,GY,1597885912,88,8.20,-59.78,76.44,2.64,Centerstone Inn & Suites - Carlyle
44,carlyle,0,US,1597885927,63,38.61,-89.37,73.00,7.34,International Lodge Motel
47,mackay,0,AU,1597885930,47,-21.15,149.20,78.80,5.82,Casa Federico B&B
65,san giovanni in fiore,0,IT,1597885937,73,39.26,16.70,73.40,3.36,Floresta Amazonica Hotel


In [66]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [67]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))